## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r'C:\Users\grime\GitHub\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,48.47,92,94,5.26,overcast clouds
1,1,Nizhniy Odes,RU,63.6445,54.8560,67.19,54,79,3.53,broken clouds
2,2,Sepatan,ID,-6.1189,106.5750,92.53,66,20,8.99,few clouds
3,3,Laguna,US,38.4210,-121.4238,66.49,81,1,4.00,clear sky
4,4,Tasiilaq,GL,65.6145,-37.6368,42.96,72,0,6.64,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 80
What is the maximum temperature you would like for your vacation? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_criteria_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
temp_criteria_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Sepatan,ID,-6.1189,106.5750,92.53,66,20,8.99,few clouds
8,8,Oktyabrskoye,RU,43.0645,44.7417,80.74,54,31,4.47,scattered clouds
16,16,Abu Samrah,SY,35.3029,37.1841,84.29,31,0,11.34,clear sky
19,19,Kirov,RU,58.5966,49.6601,87.26,38,46,8.46,scattered clouds
22,22,Padang,ID,-0.9492,100.3543,80.65,85,96,9.15,heavy intensity rain


In [6]:
# 4a. Determine if there are any empty rows.
temp_criteria_df.notnull().sum()

City_ID                166
City                   166
Country                166
Lat                    166
Lng                    166
Max Temp               166
Humidity               166
Cloudiness             166
Wind Speed             166
Current Description    166
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_df = temp_criteria_df.dropna()
cleaned_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Sepatan,ID,-6.1189,106.5750,92.53,66,20,8.99,few clouds
8,8,Oktyabrskoye,RU,43.0645,44.7417,80.74,54,31,4.47,scattered clouds
16,16,Abu Samrah,SY,35.3029,37.1841,84.29,31,0,11.34,clear sky
19,19,Kirov,RU,58.5966,49.6601,87.26,38,46,8.46,scattered clouds
22,22,Padang,ID,-0.9492,100.3543,80.65,85,96,9.15,heavy intensity rain
...,...,...,...,...,...,...,...,...,...,...
673,673,Ouallam,NE,14.3188,2.0866,89.94,42,63,15.35,broken clouds
675,675,Ishigaki,JP,24.3448,124.1572,80.60,94,75,17.27,moderate rain
676,676,Batagay-Alyta,RU,67.8006,130.4114,80.10,27,16,9.15,few clouds
678,678,Koslan,RU,63.4564,48.8989,87.89,35,16,6.20,few clouds


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sepatan,ID,92.53,few clouds,-6.1189,106.5750,
8,Oktyabrskoye,RU,80.74,scattered clouds,43.0645,44.7417,
16,Abu Samrah,SY,84.29,clear sky,35.3029,37.1841,
19,Kirov,RU,87.26,scattered clouds,58.5966,49.6601,
22,Padang,ID,80.65,heavy intensity rain,-0.9492,100.3543,
26,Butaritari,KI,83.10,scattered clouds,3.0707,172.7902,
38,Sabang,ID,82.56,light rain,5.8933,95.3214,
41,Kashi,CN,89.37,clear sky,39.4547,75.9797,
44,Nikolskoye,RU,87.80,scattered clouds,59.7035,30.7861,
47,Kampong Thum,KH,92.82,overcast clouds,12.7111,104.8887,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sepatan,ID,92.53,few clouds,-6.1189,106.5750,Jaya Imperial Park Cluster Imperial Groove
8,Oktyabrskoye,RU,80.74,scattered clouds,43.0645,44.7417,Prolesok Mini-hotel
16,Abu Samrah,SY,84.29,clear sky,35.3029,37.1841,
19,Kirov,RU,87.26,scattered clouds,58.5966,49.6601,Art Hotel
22,Padang,ID,80.65,heavy intensity rain,-0.9492,100.3543,Mercure Padang


In [13]:
hotel_df.count()

City                   166
Country                166
Max Temp               166
Current Description    166
Lat                    166
Lng                    166
Hotel Name             166
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.

cleaned_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
cleaned_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Sepatan,ID,92.53,few clouds,-6.1189,106.5750,Jaya Imperial Park Cluster Imperial Groove
8,Oktyabrskoye,RU,80.74,scattered clouds,43.0645,44.7417,Prolesok Mini-hotel
16,Abu Samrah,SY,84.29,clear sky,35.3029,37.1841,
19,Kirov,RU,87.26,scattered clouds,58.5966,49.6601,Art Hotel
22,Padang,ID,80.65,heavy intensity rain,-0.9492,100.3543,Mercure Padang
...,...,...,...,...,...,...,...
673,Ouallam,NE,89.94,broken clouds,14.3188,2.0866,
675,Ishigaki,JP,80.60,moderate rain,24.3448,124.1572,Hotel Peace Land Ishigakijima
676,Batagay-Alyta,RU,80.10,few clouds,67.8006,130.4114,
678,Koslan,RU,87.89,few clouds,63.4564,48.8989,


In [15]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

cleaned_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = cleaned_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 

locations = cleaned_hotel_df[["Lat", "Lng"]]
max_temp = cleaned_hotel_df[["Max Temp"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))